# Instalación de paqueterías

In [1]:
import ipywidgets as widgets
from IPython.display import display
from ipyfilechooser import FileChooser
from pathlib import Path
from IPython.display import HTML, display, Javascript
import xlsxwriter
import pandas as pd
import os



In [2]:
import glob
import shutil


# HTML

In [3]:
#CSS styles
waves_styles = """
<style>
body {
  margin: 0;
  padding: 0;
  box-sizing: border-box;
}
.center {
  height: 50px;
  display: flex;
  justify-content: left;
  align-items: left;
}
.wave {
  width: 3px;
  height: 30px;
  background: linear-gradient(45deg, orange, purple);
  margin: 6px;
  animation: wave 1s linear infinite;
  border-radius: 20px;
}
.wave:nth-child(2) {
  animation-delay: 0.1s;
}
.wave:nth-child(3) {
  animation-delay: 0.2s;
}
.wave:nth-child(4) {
  animation-delay: 0.3s;
}
.wave:nth-child(5) {
  animation-delay: 0.4s;
}
.wave:nth-child(6) {
  animation-delay: 0.5s;
}
.wave:nth-child(7) {
  animation-delay: 0.6s;
}
.wave:nth-child(8) {
  animation-delay: 0.7s;
}
.wave:nth-child(9) {
  animation-delay: 0.8s;
}
.wave:nth-child(10) {
  animation-delay: 0.9s;
}

@keyframes wave {
  0% {
    transform: scale(0);
  }
  50% {
    transform: scale(1);
  }
  100% {
    transform: scale(0);
  }
}
</style>
"""

#display(HTML(waves_styles))

In [4]:
#HTML Styles
wave_html = """
<div class="center">
    <div class="wave"></div>
    <div class="wave"></div>
    <div class="wave"></div>
    <div class="wave"></div>
    <div class="wave"></div>
</div>
"""

hide_waves_js = """
document.querySelector('.center').style.display = 'none';
"""

show_waves_js = """
document.querySelector('.center').style.display = 'flex';
"""

#display(HTML(wave_html))

In [5]:
#HTML loader style

loading_html = """
    <style>
    @keyframes spin {
        0% { transform: rotate(0deg); }
        100% { transform: rotate(360deg); }
    }
    .spinner {
        border: 4px solid rgba(0, 0, 0, 0.1);
        border-left: 4px solid #3498db;
        border-radius: 50%;
        width: 20px;
        height: 20px;
        animation: spin 1s linear infinite;
    }
    </style>
    <div class="spinner"></div>
    """

success_html = """
    <style>
    .icon {
        font-size: 30px;
        display: inline-block;
        width: 30px;
        text-align: center;
    }
    .success-icon {
        color: green;
    }
    </style>
    <div class="icon success-icon">&#10004;</div>
    """



<style>
    body {
        background-color: #ffdab9; 
        padding: 20px;
        margin: 0;
        ledock
    }
</style>

<div style="background: linear-gradient(to bottom right, #77216F, #6E5494); padding: 20px; font-size: 36px; color: white; font-family: 'Pacifico', cursive;">
    LeDock
</div>

Bienvenid@

El formato recomendable para el uso de LeDock es el siguiente:

- En tu carpeta personal:
    1. Ten una carpeta con el nombre de la proteína con la que se realizará el dockeo
    2. Dentro de la misma es recomendable tener la carpeta `/mol2` generada
    3. Tener el archivo: `nombre_de_la_proteina.pdb`


In [6]:
# Obtener el directorio actual
actual_dir = %pwd
# Definir la variable global para almacenar la proteína
global protein_name
# Nombres por defecto para archivos
pro = "pro.pdb"
dock_in = "dock.in"

# Ruta ejecutable
lepro = "/home/jarv11/share/apps/lepro/1.0/bin/lepro"

# Configuración de botones interactivos 
pro_button = widgets.Button(description="procesar proteína", button_style = 'Warning', tooltip = 'Iniciar procesamiento de la proteína antes de dockear')
dock_button = widgets.Button(description = "Dockear")
main_button = widgets.Button(description = 'Dockear',disabled = False ,button_style = 'warning' ,tooltip = 'Iniciar proceso de dockeo una vez que se haya procesado la proteína',icon = 'play')
csv_button = widgets.Button(description = 'Generar .csv', disabled = False, button_style = 'success', tooltip = 'Generar archivo .csv automáticamente')

In [7]:
# Obtener la ruta del directorio 
home = str(Path.home()) 

# Configurar selectori de archivos
folder_prote = FileChooser(f'{home}/') 
folder_prote.title = '<b>Selecciona la proteína<b>'
folder_prote.show_only_dirs = False;
folder_prote.filter_pattern = '*.pdb'
display(folder_prote)

# selección de la proteína
def select_proteina():

    global proteina, lepro_convert, proteina_folder
    proteina = folder_prote.selected # obtener nombre de la proteína 
    proteina_folder = proteina.split('/')[-2]
    
    lepro_convert = f"/home/jarv11/share/apps/lepro/1.0/bin/lepro {proteina}"
    
folder_prote.register_callback(select_proteina) 

FileChooser(path='/home/jarv11', filename='', title='<b>Selecciona la proteína<b>', show_hidden=False, select_…

In [9]:
# Actualizar el archivo de configuración (dock.in)
# El que tiene los parametros entiendo
def update_in(input, new_value):
    
    with open(input, "r+") as file:
        lines = file.readlines()
        file.seek(0)
        
    with open(input, "w") as file:
        for i, line in enumerate(lines):
            if "Receptor" in line:
                lines[i + 1] = new_value+"\n"
            if "Number of binding poses" in line:
                lines[i + 1] = "20"+"\n"
            if "Ligands list" in line:
                lines[i + 1] = "ligands.txt"+"\n" # Lista de moléculas en .mol2 y su ID
        file.writelines(lines)
        file.truncate()

In [10]:
global parent

# Selección de directorios 

def dir_selector(proteina, opt):
    
    parent = os.path.dirname(proteina)
    mol2_dir = os.path.join(parent, "mol2")
    dok_dir = os.path.join(parent, "dok")
    
    if opt == 1:
        return parent
    if opt == 2:
        return mol2_dir
    elif opt == 3:
        return dok_dir
        

In [11]:
# Mover archivos a la carpeta (mol2)
def move_to_mol2(input, protein_pro):
    
    input_path = f"{os.path.dirname(proteina)}/{input}"
    protein_pro_path = f"{os.path.dirname(proteina)}/{protein_pro}"
    temp_path = f"{os.path.dirname(proteina)}/mol2"

    !mv {input_path} {temp_path}/
    !mv {protein_pro_path} {temp_path}/

In [12]:
# Crea la carpeta dok (directorio parent)
def create_dir():
    parent_folder = dir_selector(proteina, 1)
    !mkdir {parent_folder}/dok

In [13]:
def write_score(folder_path, parent):
    start = "REMARK Cluster" #linea para encontrar los scores
    dict_score = read_score(folder_path, start) # Lee los scores
    sorted_dict = sorted(dict_score.items(), key=lambda x: x[1]["score"]) #Ordenar los scores

    output_file_path = os.path.join(parent, "output_score.txt")

    with open(output_file_path, 'w') as output_file:
        output_file.write("Dockeo por Score:\n")
        for file_name, data in sorted_dict:
            file_name_f = os.path.splitext(file_name)[0]
            output_file.write(f"{file_name_f}: {data['score']}\n")
            print(f"{file_name_f}: {data['score']}\n")
    print("Se ha escrito correctamente en el archivo o")

In [14]:
def read_score(folder_path, start):
    file_score_dict = {}
    
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        
        if os.path.isfile(file_path):
            with open(file_path, 'r') as file:
                for line in file:
                    if line.startswith(start): #busca la linea clave, start
                        score_unit = line.split("Score: ")[1].split()[0]
                        score = float(score_unit.replace("kcal/mol", "")) # Extraer los scores
                        line_no_prefix = line.split("Score: ")[1].strip()
                        file_score_dict[file_name] = {"line": line_no_prefix .strip(), "score": score}
                        break
    return file_score_dict

In [15]:
def read_txt():

    ligands = [] 
    score = []

    list = []

    os.chdir(dir_selector(proteina, 1)) # cambiar el directorio 
    
    file_path = os.path.join(os.getcwd(), "output_score.txt") # Ruta del archivo

    # Ver archivo output
    with open(file_path,'r') as file:
        next(file)
        for line in file:
            list = line.split(": ")
            ligands.append(list[0])
            score.append(list[1].strip())
            
    return ligands, score

#### Estatus del procesamiento de la proteína

In [16]:
global ledock_cmd

lepro_status = widgets.Output()
display(lepro_status)

def lepro():

    with lepro_status:
        lepro_status.clear_output()
        display(HTML(waves_styles))
        display(HTML(wave_html))
    
    global input_dock

    protein_name = os.path.basename(proteina)
    protein_pro = protein_name.replace('.pdb', '_pro.pdb')
    input_dock = protein_name.replace('.pdb', '')
    input_dock += ".in"

    os.chdir(os.path.dirname(proteina))
    
    !{lepro_convert}
    
    os.rename(pro, protein_pro)
    os.rename(dock_in, input_dock)
    
    update_in(input_dock, protein_pro)
    move_to_mol2(input_dock, protein_pro)

    with lepro_status:
            lepro_status.clear_output()
            display(HTML(success_html))
            print("Revisa los archivos .in y _pro.pdb generados en tu carpeta mol2")

Output()

#### Estatus del dockeo

(Las moléculas dockeadas se encuentran en la carpeta "dok" generada automáticamente)

In [17]:
import os
import glob
import shutil
import time
import threading
from IPython.display import display, clear_output
import ipywidgets as widgets


In [18]:
ledock_status = widgets.Output()
display(ledock_status)

def monitor_dock_files(dok_dir, stop_event):
    existing = set()
    with ledock_status:
        clear_output(wait=True)
        display(widgets.HTML("<b>Ejecutando LeDock...</b>"))
        output_area = widgets.Output()
        display(output_area)

    while not stop_event.is_set():
        current_files = set(f for f in os.listdir(dok_dir) if f.endswith('.dok'))
        new_files = current_files - existing
        if new_files:
            with output_area:
                for file in new_files:
                    print(f"Archivo generado: {file}")
            existing.update(new_files)
        time.sleep(2)

def ledock_live():
    create_dir()

    mol2_dir = dir_selector(proteina, 2)
    dok_dir = dir_selector(proteina, 3)
    parent_dir = dir_selector(proteina, 1)

    os.makedirs(dok_dir, exist_ok=True)
    os.chdir(mol2_dir)

    ledock_cmd = f"/home/jarv11/share/apps/ledock/1.0/bin/ledock {input_dock}"

    stop_event = threading.Event()
    monitor_thread = threading.Thread(target=monitor_dock_files, args=(dok_dir, stop_event))
    monitor_thread.start()

    os.system(ledock_cmd)

    stop_event.set()
    monitor_thread.join()

#  Verificación de posibles ubicaciones
    possible_dirs = [mol2_dir, parent_dir, os.getcwd()]

    for d in possible_dirs:
        for file in glob.glob(os.path.join(d, "*.dok")):
            try:
                shutil.move(file, dok_dir)
                print(f"Movido: {os.path.basename(file)} a {dok_dir}")
            except Exception as e:
                print(f"Error al mover {file}: {e}")

    write_score(dok_dir, parent_dir)

    with ledock_status:
        display(widgets.HTML("<b style='color:green'>¡Proceso finalizado! Se ha generado el archivo output_score.txt</b>"))

'''  
    for file in glob.glob(f"{mol2_dir}/*.dock"):
        shutil.move(file, dok_dir)

    write_score(dok_dir, parent_dir)

    with ledock_status:
        display(widgets.HTML("<b style='color:green'>¡Proceso finalizado! Se ha generado el archivo output_score.txt</b>"))
'''



Output()

'  \n    for file in glob.glob(f"{mol2_dir}/*.dock"):\n        shutil.move(file, dok_dir)\n\n    write_score(dok_dir, parent_dir)\n\n    with ledock_status:\n        display(widgets.HTML("<b style=\'color:green\'>¡Proceso finalizado! Se ha generado el archivo output_score.txt</b>"))\n'

In [19]:
'''
ledock_status = widgets.Output()
display(ledock_status)

def ledock():

    with ledock_status:
        ledock_status.clear_output()
        display(HTML(waves_styles))
        display(HTML(wave_html))
        
    
    create_dir()
    
    mol2_dir = dir_selector(proteina, 2)
    dok_dir = dir_selector(proteina, 3)
    parent_dir = dir_selector(proteina, 1)
    
    os.chdir(mol2_dir)

    # /home/jarv11/share/apps/ledock/1.0/bin/ledock
    ledock = f"/home/jarv11/share/apps/ledock/1.0/bin/ledock {input_dock}"
    !{ledock}

    !mv {mol2_dir}/*.dock {dok_dir}/

    write_score(dir_selector(proteina, 3), dir_selector(proteina, 1))

    with ledock_status:
        ledock_status.clear_output()
        display(HTML(success_html))
        print("Está disponible el archivo output_score.txt con los nombres de las moléculas y el score obtenido")

SyntaxError: EOF while scanning triple-quoted string literal (900688033.py, line 32)

#### Estatus del archivo .csv

In [20]:
csv_status = widgets.Output()
display(csv_status)

def write_csv():

    with csv_status:
        csv_status.clear_output()
        display(HTML(waves_styles))
        display(HTML(wave_html))

    import csv
    
    os.chdir(dir_selector(proteina, 1))
    ligands, scores = read_txt()
    
    with open('output.csv', 'w', newline='') as file:
        csvwriter = csv.writer(file)
        for ligand, score in zip(ligands, scores):
            csvwriter.writerow([ligand, score])

    with csv_status:
        csv_status.clear_output()
        display(HTML(success_html))
        print("Está disponible el archivo output.csv con los nombres de las moléculas y el score obtenido")

Output()

In [21]:
def main():
    display(pro_button)
    pro_button.on_click(lambda _: lepro())

    def run_ledock(b):
        main_button.disabled = True
        ledock_live()
        main_button.disabled = False

    display(main_button)
    main_button.on_click(run_ledock)

    display(csv_button)
    csv_button.on_click(lambda _: write_csv())


In [22]:
'''
def main():

    display(pro_button)
    pro_button.on_click(lambda _: lepro())
    
    display(main_button)
    main_button.on_click(lambda _: ledock_live())

    display(csv_button)
    csv_button.on_click(lambda _: write_csv())


SyntaxError: EOF while scanning triple-quoted string literal (1826338434.py, line 11)

In [23]:
main()

Button(button_style='warning', description='procesar proteína', style=ButtonStyle(), tooltip='Iniciar procesam…

Button(button_style='warning', description='Dockear', icon='play', style=ButtonStyle(), tooltip='Iniciar proce…

Button(button_style='success', description='Generar .csv', style=ButtonStyle(), tooltip='Generar archivo .csv …

In [25]:
mol2_dir = dir_selector(proteina, 2)
dok_dir = dir_selector(proteina, 3)
parent_dir = dir_selector(proteina, 1)

print("Verificando archivos .dok después del docking...\n")

possible_dirs = [mol2_dir, parent_dir, os.getcwd()]

for d in possible_dirs:
    print(f"Buscando en: {d}")
    archivos = glob.glob(os.path.join(d, "*.dok"))
    print(f"→ Encontrados: {len(archivos)} archivos")

    for file in archivos:
        print(f"Intentando mover: {file}")
        try:
            shutil.move(file, dok_dir)
            print(f"✅ Movido: {os.path.basename(file)} a {dok_dir}")
        except Exception as e:
            print(f"⚠️ Error al mover {file}: {e}")


Verificando archivos .dok después del docking...

Buscando en: /home/jarv11/Docking/UpLedock/PruebasCodeNew/4XZH_STRAT/mol2
→ Encontrados: 0 archivos
Buscando en: /home/jarv11/Docking/UpLedock/PruebasCodeNew/4XZH_STRAT
→ Encontrados: 0 archivos
Buscando en: /home/jarv11/Docking/UpLedock/PruebasCodeNew/4XZH_STRAT
→ Encontrados: 0 archivos


In [94]:
#write_score(dok_dir, parent_dir)

In [3]:
import os
import glob

def extract_scores(dok_folder, output_file='output_score.txt'):
    results = []

    for dok_file in glob.glob(os.path.join(dok_folder, '*.dok')):
        with open(dok_file, 'r') as f:
            for line in f:
                if line.startswith('REMARK') and 'Binding Energy' in line:
                    parts = line.strip().split('=')
                    if len(parts) == 2:
                        score = parts[1].split()[0]  # obtiene el número
                        name = os.path.basename(dok_file)
                        results.append((name, score))
                    break  # ya no seguimos leyendo ese archivo

    # Guardar el archivo
    with open(os.path.join(dok_folder, output_file), 'w') as out:
        for name, score in results:
            out.write(f"{name}\t{score}\n")

    print(f"✅ Archivo de resultados generado: {output_file}")

# Usa la función con tu carpeta
extract_scores('/home/jarv11/Docking/UpLedock/PruebasCodeNew/4XZH_STRAT/dok-7501to10006')  # o cambia por la ruta correcta


✅ Archivo de resultados generado: output_score.txt


In [4]:
import os
import glob

def extract_scores(dok_folder, output_file='output_score.txt'):
    results = []

    for dok_file in glob.glob(os.path.join(dok_folder, '*.dok')):
        with open(dok_file, 'r') as f:
            for line in f:
                if "Score:" in line:
                    try:
                        score = line.strip().split("Score:")[1].split()[0]
                        name = os.path.basename(dok_file)
                        results.append((name, score))
                    except IndexError:
                        print(f"⚠️ No se pudo extraer score de {dok_file}")
                    break  # solo necesitamos una línea por archivo

    # Guardar el archivo
    output_path = os.path.join(dok_folder, output_file)
    with open(output_path, 'w') as out:
        for name, score in results:
            out.write(f"{name}\t{score}\n")

    print(f"✅ Archivo de resultados generado: {output_path}")

# Usa tu ruta
extract_scores('/home/jarv11/Docking/UpLedock/PruebasCodeNew/4XZH_STRAT/dok-7501to10006')


✅ Archivo de resultados generado: /home/jarv11/Docking/UpLedock/PruebasCodeNew/4XZH_STRAT/dok-7501to10006/output_score.txt
